# Generate RSI using daily data service - Python

### Overview
*The Relative Strength Index*, developed by J. Welles Wilder measures the speed and change of price movements. *RSI* oscillates between zero and 100. Traditionally the RSI is considered:
* overbought when above 70
* oversold when below 30

This sample demonstrates how to request and plot **on-demand** *Relative Strength Index - RSI* from a `daily data service`.

### Inputs/outputs
RSI's request require instrument's identifier, date time intervals and number of periods as per inputs.

$$ RSI = 100 - \frac{100}{1 + RS} $$
Where:
* $RS = \frac{AvgU}{AvgD}$ is the *relative strength*
* *AvgU* average of last N *close prices* up moves
* *AvgD* average of last N *close prices* down moves
* *N* is the period of the RSI

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve daily data from the hosted service. The queried endpoint in this script are:
* *DailyRsiService*: to directly retrieve daily data from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.daily_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matplotlib* as per display package

***

# Run daily RSI sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly matplotlib systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.daily_analytics.v1.daily_rsi_pb2 as daily_rsi
import systemathics.apis.services.daily_analytics.v1.daily_rsi_pb2_grpc as daily_rsi_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data
To request *daily rsi* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The RSI parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = "AAPL"
exchange = "XNGS"

#### 3.2 RSI parameters

In [ ]:
# set the RSI window length
window = 14;

#### 3.3 Time period selection

In [ ]:
# set data interval (we are using Google date time format) 
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2019, month = 1, day = 1),
    end_date = date.Date(year = 2020, month = 12, day = 31)
)

# generate constraints based on the previous date selection
# time constraints are not neeeded for the dividends
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

#### 3.4 RSI request creation
The following code snippet creates *gRPC client*, process daily RSI requests and returns the reply.

In [ ]:
# generate daily rsi request
request = daily_rsi.DailyRsiRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker), 
    length = window,
    constraints = constraints
)

In [ ]:
try:
    # open a gRPC channel
    with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
        credentials = grpc.ssl_channel_credentials(f.read())
    with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:
        
        # instantiate the daily rsi service
        service = daily_rsi_service.DailyRsiServiceStub(channel)
        
        # Process the daily long rsi request
        response = service.DailyRsi(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

### Step 4: Visualize data

#### 4.1 Retrieve RSI data
In the following code snippets, the daily RSI reply is reprocessed in a dataframe in order to visualize the results:

In [ ]:
# prepare the RSI dataframe
dates=[datetime(r.date.year, r.date.month, r.date.day) for r in response.data]
rsi_data = [r.rsi for r in response.data]
prices = [r.value for r in response.data]

In [ ]:
# create a pandas dataframe with dates, prices and rsis
d = {'Date': dates, 'Rsi': rsi_data, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot RSI and daily prices with matplotlib

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(25,10))
price = ax.plot('Date', 'Price', data=df, marker='', color='blue', linewidth=2)

# twin x-axis for two different y-axis
ax2=ax.twinx()
rsi = ax2.plot('Date', 'Rsi', data=df, marker='', color='orange', linewidth=2)

# build common legend including all graphs
graphs = price + rsi
labels = [g.get_label() for g in graphs]
ax.legend(graphs, labels)

# set graph title and axis label
ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="blue",fontsize=14)
ax2.set_ylabel("Rsi",color="orange",fontsize=14)
plt.title('RSI and daily prices for {0}-{1}'.format(ticker, exchange))

plt.axhline(y=70, color='r', linestyle='-')
plt.axhline(y=30, color='g', linestyle='-')
plt.show()

#### 4.3 Plot RSI and daily prices with plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# add traces for daily rsi and prices
fig.add_trace(go.Scatter(x=df['Date'], y=df['Rsi'], name='{0}-days Rsi'.format(window)), secondary_y=False,)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price'), secondary_y=True,)

# Add figure title
fig.update_layout(title = 'RSI and daily prices for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Rsi', secondary_y=False)
fig.update_yaxes(title_text = 'Price', secondary_y=True)

# add overbought and oversold lines
fig.add_hline(y=70, line=dict(color="Green"))
fig.add_hline(y=30, line=dict(color="Red"))

fig.show()